IDEAS/ THINGS TO TRY FOR FINDING DIAGNOSIS

- look for renal food purchase
- look in 'Assessment' for diagnosis
- have a look at everything with renal/kidney (e.g. we don't want acute kidney disease)
- look for CKD meds e.g. Semintra in Item Label, Item Name, Examination Text
- Goal: we need to find the TIME at which a cat is diagnosed renal
- PLAN:  pull everything into columns, then go through manually

In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
columns = ['Database Name', 'Patient Number', 'Client Number', 'Consultation Number', 'Item Name', 'Units',
           'Item Label', 'Clinic Code', 'Consulting Vet', 'Patient Desexed', 'Deceased Date',
           'Visit Date', 'Consultation Date', 'Examination Text' ,'Patient Species', 'Patient Breed',
           'Patient Colour', 'Patient Sex', 'Patient Date of Birth', 'Insured', 'weight', 'temperature',
           'HeartRate', 'BodyScore', 'DentalGrade', 'RespRate', 'PainScore', 'BP', 'CRT', 'MMColour']

In [3]:
vetcompass_df = pd.read_csv("../Data/vet_compass/STR024A03 20190719 sample x10,000.csv", header=None, names = columns)

/Users/ruth/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,2,3,19,20,21,22,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
vetcompass_df.head()

,Database Name,Patient Number,Client Number,Consultation Number,Item Name,Units,Item Label,Clinic Code,Consulting Vet,Patient Desexed,Deceased Date,Visit Date,Consultation Date,Examination Text,Patient Species,Patient Breed,Patient Colour,Patient Sex,Patient Date of Birth,Insured,weight,temperature,HeartRate,BodyScore,DentalGrade,RespRate,PainScore,BP,CRT,MMColour
0,RxNSW,7138614,NaN,13869415,NaN,NaN,NaN,1,WA1,1,NaN,2012-07-05 17:09:03.000,2012-07-05 17:09:03.000,-------------------- Fees Charged: -----------...,Cat,Persian,White,Female,2010-07-05 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RxNSW,7737653,NaN,22925726,NaN,NaN,NaN,1,ZOU,1,NaN,2003-04-25 00:00:00.000,2003-04-25 00:00:00.000,NaN,Cat,Tonkinese,Lilac,Male,2002-11-26 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RxNSW,7738120,NaN,22934344,NaN,NaN,NaN,1,ZOU,1,NaN,2004-01-10 00:00:00.000,2004-01-10 00:00:00.000,NaN,Cat,Domestic Medium Hair,Tortoiseshell,Female,2003-02-14 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RxNSW,7745004,NaN,23043653,NaN,NaN,NaN,1,ZOU,1,NaN,2013-02-18 16:47:52.000,2013-02-18 16:47:52.000,NaN,Cat,Siamese,Lilac Point,Male,2008-07-20 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RxNSW,7742894,NaN,23043708,NaN,NaN,NaN,1,ZOU,1,2013-11-05 10:00:34.000,2013-02-20 08:40:38.000,2013-02-20 08:40:38.000,Dental; Renal Dz,Cat,Domestic Long Hair,Ginger & White,Male,1994-03-16 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
vetcompass_df['CKD_indicated'] = False

In [5]:
def get_patterns(df, pattern):
    # for loop of each exmination text in df as long as it is strings, re.findall: returns all matches of pattern in the string
    note_captions_lists = [re.findall(string=text, pattern=pattern) for text in df['Examination Text'] if isinstance(text, str)]
    note_captions = [text for text_list in note_captions_lists for text in text_list]
    note_captions_set = set(note_captions)
    return note_captions_set

# Look for mentions of CKD/kidney/renal

In [9]:
renal_string = "CKD.{1,150}|ckd.{1,150}|Kidney.{1,150}|kidney.{1,150}|renal.{1,150}|RENAL.{1,150}"

# try using the function
len(get_patterns(vetcompass_df, renal_string))

6709

In [18]:
get_patterns(vetcompass_df, renal_string)

{'ckdate requested by Cindy - processed by Meg @ HPP 31/01/14 Vaccination = $85.70 + Removed subsequent pet  discount',
 'renal  parameters.',
 'Kidneys palpate ok. Laboratory: Assessment: bloods for wellness. Treatment: comfortis for fleas. Plan:await bloods VETNOSTICS Hb 147 g/L (80-140) WBC 5.9 x1',
 'kidney was enlarged Started eating again after treatment The past 4 days - inappetent Vomiting or Diarrhoea: v+ intermittent; no d+ V+ today Diet: fancy fea',
 'renal disease - vacc no longer  recommended/required)',
 'kidneys; T4 and WBC all WNL O asked about previous vet (LL ) saying she had small kidneys- advised different for each individual vet on palpation; but I cou',
 'renal diet QAR; IVF running OK HR: 160 grade 4/6 murmur (auscultated outside cage = increased stress); RR: 22 clear chest; MM: pink; CRT: < 2sec; Temp: 38.',
 'CKD Hyperthyroidism Diabetes mellitus Osteoarthritis Urinary tract neoplasia Decreasing body weight Reduced urine specific gravity Any cat with a history'

In [ ]:
# returns only the rows in the df where examination text includes the words in renal_string
searched_df = vetcompass_df[vetcompass_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern=renal_string)) if isinstance(text, str) else False)]

In [17]:
len(searched_df)

32808

In [15]:
len(searched_df['Patient Number'].unique())

2010

In [22]:
searched_df[['Examination Text', 'Patient Number']].iloc[100]

Examination Text    Reason: 615- R/c Appointment Notes: Overdue re...
Patient Number                                                6067233
Name: 4664, dtype: object

# Suggestion
- maybe we need to do this by hand for the 33808
- pupd meand weeing a lot, drinking a lot --> indictes CKD
- ddx means differential diagnosis
- train an NLP model to understand diagnosis?  e.g. sentiment analysis?

In [30]:
this_cat = vetcompass_df[vetcompass_df['Patient Number']==6067233]
len(this_cat)

185

In [27]:
for index, row in this_cat.iterrows():
     print(row['Examination Text'])
     print()

Reason: transfer from AEC Appointment Notes: Overdue reminders History: been to AEC overnight pcv 25 % done at pm measures blood pressure 200 so hypertensive ? Re anaemia separate problem started on norvasc 1/4 p.O. Examination: Still pale mm eaten samll amt a home hydrated admit to observe Laboratory: Assessment: Treatment: Plan: drank water quiet eaten small amt food bloods taken last night sent for urgent pcv here 27 so improved slightly pm 1/2 norvasc p.O. Keep and await reults if o ok o elected to take home will ring re path +1 29/9 GJT spoke O advised t4 normal does have azotaemia so poss anaemia + bp issues related to renal failure Advised collect urine O booked to recheck bp + collect further meds with NC on Saturday will try and collect urine to bring in then Total T4 13 nmol/L (6-52) Vetnostics North Ryde NATA Accreditation Number: 14599 Hb 80 g/L (80-140) WBC 16.7 x109/L (6.0-16.0) RCC 5.7 x1012/L (5.5-10.0) Neut 13.4 x109/L (3.8-10.1) Hct 0.24 (0.28-0.45) Lymp 2.3 x109/L (1

# Can we just look for diagnosis?

- create a column of both 'diagnosis' and 'differential diagnosis' seperately

In [32]:
diagnosis_string = 'Diagnosis:'

In [33]:
diagnosis_df = vetcompass_df[vetcompass_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern=diagnosis_string)) if isinstance(text, str) else False)]

In [35]:
len(diagnosis_df)

18663

In [36]:
len(diagnosis_df['Patient Number'].unique())

946

In [40]:
diagnosis_df['Examination Text'].iloc[100]

'Reason: Cu Eye Appointment Notes: rw History: Owner has noticed discharge from left eye. Otherwise well. EDDU normally. No V or D. Not coughing or sneezing. No nasal discharge noted. Vaccinations UTD. Exam: Watery ocular discharge R eye and mild blepharospasm. PLR normal. Menace and palpebral normal. Conjunctiva erytheamtous and mildly swollen. Alcain drops and fluorescein stain applied. No uptake. Diagnosis: Conjunctivitis L eye Treatment: Opticin TID for 5-7 days. Discussed possibility of flare up of cat flu if had it as a young kitten and advised to ensure to be vigilant about vaccinations and reducing stress. --- 09/11/2017 09:22:14 AM AT1: Examinations - 03/11/2017 11:25:20 AM PPF Deferred: no hx Resolved - sent text  TA'

In [43]:
def make_regex(values):
    return f'({values})[\s\:\=\-]*(.*?)\:'


def add_column(df, values, colname):
    df[colname] = df['Examination Text'].str.extract(make_regex(values), expand=True, flags=re.IGNORECASE)[1]
    return df

In [66]:
columns_to_add = {
    'diagnosis': 'Differential\sDiagnosis\:\sDiagnosis|differential\sdiagnosis|diagnosis',
    'history': 'history'}

In [67]:
for column_name in columns_to_add.keys():
    print("processing: "+ column_name + " with pattern " + columns_to_add[column_name])
    vetcompass_df = add_column(vetcompass_df, columns_to_add[column_name], column_name)
    print('\x1b[32m' + str(len(vetcompass_df[column_name].unique())) + " unique values found for " + column_name + '\x1b[0m')

processing: diagnosis with pattern Differential\sDiagnosis\:\sDiagnosis|differential\sdiagnosis|diagnosis
2104 unique values found for diagnosis
processing: history with pattern history
37756 unique values found for history


In [65]:
vetcompass_df['Diagnosis'].dropna().sample(10)

633807    ; blood loss although not sure where; destruct...
390317                                            Diagnosis
719361                                                Tests
572695                                            Diagnosis
570577                                            Diagnosis
669450                                            Diagnosis
670247                                            Diagnosis
719834                                        Problem\~list
568832                                            Diagnosis
123027                                           Assessment
Name: Diagnosis, dtype: object

In [62]:
vetcompass_df.iloc[654975]['Examination Text']

'Reason: Off Colour Drinking More KD History: Has increased drinking Has had diarrhoea as well. Is outside cat thought. No blood in it No v+ Still eating well Progressive loss in weight over the last couple of months Still seems happy in herself Being fed kitten food to try and put weight onto her Examination: BAR BCS: 2/9 Oral: mm pink; crt < 2 seconds; moist. DS 4/4 R maxillary PM4 and molar 1 cannot actually see tooth for the tartar LN: wnl Thyroid gland: palpable Cardio: systolic heart murmur; femoral pulse not that prominent. Jugular pulse distended but not pulsing Resp: wnl Perineum: nad Vulva: nad Integument: seemed mostly well kept M.S: severe muscle atrophy/wastage over all body Abdo: nothing abnormal noted apart from just being severely underweight Differential diagnosis: Thyroid; renal; DM; neoplasia; IBD; pancreas issue Assessment: advised bloods is required to determine what is going on Diarrhoea could be secondary to anything--thyroid; IBD; neoplasia; primary gut issue? A

# Look at history 

In [69]:
vetcompass_df['history'].dropna().sample(10)

458862                 Doing well at home; EDDU Examination
600720    edud well. no v+ d+ coughing and sneezing. No ...
604728        No health concerns at the momment Examination
373729    eating/drinking ok; no v/d. No urinary probs r...
234917    O is quite distressed as she saw Snowball get ...
708321                                          Examination
256812                                          Examination
117717    Has been all OK at home. Is looked after by th...
127230                                          Examination
677284                                               Update
Name: history, dtype: object

In [70]:
renal_history_df = vetcompass_df[vetcompass_df['history'].apply(lambda text: bool(re.search(string=text, pattern=renal_string)) if isinstance(text, str) else False)]

In [71]:
len(renal_history_df)

3879

In [72]:
len(renal_history_df['Patient Number'].unique())

321

In [82]:
renal_history_df['history'].iloc[270]

"Has been very quiet lately and possibly also did a V+ the other day. Not eating much and this morning she didn't anything. She does come for food but seems to back off her food. Also have bad teeth and CKD; OA. K/D wet food only. Also has spots on her nose and a bit weepy around the eyes. Has come and gone in the past and normally a clean up with saline settles it. Weight loss with 500 grams since last visit Skin tent slow. Sunken in eyes Cannot feel enlarged thyroid on palpation Small bladder; could not get urine Laboratory"

# Try looking for renal food purchase

In [90]:
food_string = "food.{1,150}|kd.{1,150}|renal.{1,150}|k\/d.{1,150}"

In [91]:
food_df = vetcompass_df[vetcompass_df['Item Name'].apply(lambda text: bool(re.search(string=text, pattern=food_string, flags=re.IGNORECASE)) if isinstance(text, str) else False)]

In [85]:
vetcompass_df
vetcompass_df[['Item Name', 'Item Label']].sample(10)

,Item Name,Item Label
53979,RCW Dog Renal 150g (10),NaN
21014,Reminder Vax 12mth,NaN
521851,Milbemax Cat 2 - 8Kg,NaN
253258,Reminder 12mth,NaN
639690,!Miscellaneous Preventatives,NaN
84969,Anesthesia Fees,NaN
372326,RES-Exam Room,NaN
86556,Boarding,NaN
91319,!Miscellaneous Merchandise,NaN
370699,Hills Cat C/D M/Care 3.8Kg,NaN


In [98]:
food_df['Item Name'].sample(10)

355542                     RCW Cat Renal-Tuna 85 (12)
67755                RCW Cat Renal - Chicken 85g (12)
560908                            Hills Cat K/D 1.8Kg
713448                 RCW Cat Renal 85g Chicken (12)
512876                              RCW Cat Renal 4Kg
578420                    RCW Cat Renal 85g Tuna (12)
57633     Hills Cat K/D Chicken And Veg Stew 82g (24)
188037               RCW Cat Renal - Chicken 85g (12)
460622                              RCW Cat Renal 2Kg
318197                        RCW Cat Renal 100g (12)
Name: Item Name, dtype: object

In [96]:
len(food_df)

7148

In [97]:
len(food_df['Patient Number'].unique())

626

# Try medication
- phosphate binders (alutab)
- anti nausea (e.g. maxalon)
- semintra (for the treatment of protein urea in cats)
- fluid treatment 